In [1]:
import numpy as np
import pandas as pd
import torch
import warnings
from datasets import load_dataset, Dataset
from utils.database import CreateDatabase, SearchDatabase
from utils.model_rag import ModelRag
from transformers import pipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')

In [2]:
path_client = 'aep_chat_bot'
collection_name = 'regulatory_documentation'

# Dataset

In [3]:
total = pd.read_excel('data/total.xlsx', dtype={'point': str})
# np_001_15 = np_001_15.rename(columns={'НТД': 'document', 'Пункт': 'point', 'Описание': 'description'})
dataset = Dataset.from_pandas(total)

In [4]:
dataset[0]

{'document': 'НП-005-16',
 'point': '1',
 'description': 'Настоящие федеральные нормы и правила в области использования атомной энергии "Положение о порядке объявления аварийной обстановки, оперативной передачи информации и организации экстренной помощи атомным станциям в случаях радиационно опасных ситуаций" (далее - Положение) разработаны в соответствии со статьей 6 Федерального закона от 21 ноября 1995 г. N 170-ФЗ "Об использовании атомной энергии" (Собрание законодательства Российской Федерации, 1995, N 48, ст. 4552; 1997, N 7, ст. 808; 2001, N 29, ст. 2949; 2002, N 1, ст. 2; N 13, ст. 1180; 2003, N 46, ст. 4436; 2004, N 35, ст. 3607; 2006, N 52, ст. 5498; 2007, N 7, ст. 834; N 49, ст. 6079; 2008, N 29, ст. 3418; N 30, ст. 3616; 2009, N 1, ст. 17; N 52, ст. 6450; 2011, N 29, ст. 4281; N 30, ст. 4590, ст. 4596; N 45, ст. 6333; N 48, ст. 6732; N 49, ст. 7025; 2012, N 26, ст. 3446; 2013, N 27, ст. 3451), постановлением Правительства Российской Федерации от 1 декабря 1997 г. N 1511 "Об

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Create Database

In [6]:
# embedding_model = SentenceTransformer(
#     "intfloat/multilingual-e5-large",
#     model_kwargs={'torch_dtype': torch.float16}
# )

# create_db = CreateDatabase(
#     embedding_model=embedding_model,
#     dataset=dataset,
#     device=device,
#     collection_name=collection_name,
#     path=path_client    
# )

# create_db.create()

# Generation

In [7]:
generation_pipeline = pipeline(
    'text-generation',
    # model='Qwen/Qwen2-1.5B-Instruct',
    model='train_model/Qwen_Qwen2_1.5B_Instruct',
    device=device,
    torch_dtype=torch.float16
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [8]:
embedding_model = SentenceTransformer(
    "intfloat/multilingual-e5-large",
    model_kwargs={'torch_dtype': torch.float16}
)

In [9]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': device}
)

In [22]:
model = ModelRag(
    generation_pipeline=generation_pipeline,
    embedding_model=embedding_model,
    cross_encoder=cross_encoder,
    path_client=path_client,
    device=device,
    collection_name=collection_name,
    temperature=0.8,
    top_p=0.6,
    max_new_tokens=1024
)

RuntimeError: Storage folder aep_chat_bot is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.

In [31]:
query = 'Какие бывают категории сейсмостойкости и что к ним относится?'
answer, context, history = model.predict(query)
print(answer)

Категории сейсмостойкости в атомной энергетике определяются по уровню ответственности за обеспечение безопасности и работоспособности после прохождения землетрясения. Они разделяются на три категории:

1. Класс I сейсмостойкости - это самые безопасные категории, где системы и элементы, которые обеспечивают работу станции, могут быть полностью отключены в случае землетрясения. 

2. Класс II сейсмостойкости - эти системы и элементы могут быть частично отключены, но они должны продолжать функционировать после землетрясения.

3. Класс III сейсмостойкости - это менее безопасные категории, где системы и элементы могут быть полностью отключены или частично отключены в случае землетрясения.

Важно отметить, что эти категории не являются абсолютными и могут меняться в зависимости от специфической ситуации и требований проекта.


In [30]:
# context.split('Название')

['',
 ': НП-031-01. Пункт: Здания, сооружения, строительные конструкции и основания, технологическое и электротехническое оборудование, трубопроводы, приборы, другие системы и элементы АС в зависимости от степени их ответственности для обеспечения безопасности при сейсмических воздействиях и работоспособности после прохождения землетрясения должны быть классифицированы на три категории сейсмостойкости с учетом их класса безопасности согласно требованиям Общих положений обеспечения безопасности атомных станций: ; ',
 ': НП-036-05. Пункт: Здания, сооружения, строительные конструкции и основания, технологическое и электротехническое оборудование, трубопроводы, приборы, другие системы и элементы АС в зависимости от степени их ответственности для обеспечения безопасности при сейсмических воздействиях и работоспособности после прохождения землетрясения должны быть классифицированы на три категории сейсмостойкости с учетом их класса безопасности согласно требованиям Общих положений обеспечени

In [13]:
# query = 'Привет! Давай знакомиться?'
# answer, context, history = model.predict(query)
# print(answer)
# history

In [14]:
# query = 'Какого цвета небо?'
# answer, context, history = model.predict(query)
# print(answer)
# history

In [15]:
# query = 'А почему именно такой цвет?'
# answer, context, history = model.predict(query)
# print(answer)
# history

In [16]:
# query = 'Опиши четыре класса безопасности по НП-001-15. Расскажи подробно что они включают и какие эелементы к ним относятся? В каком пункте НП-001-15 идет их описание?'
# answer, context = model.predict(query)
# print(answer)

In [17]:
# query = 'Расскажи про 5 уровней глубокоэшелонированной защиты'
# answer, context = model.predict(query)
# print(answer)

In [18]:
# query = 'Перечисли основные функции систем безопасности на атомной станции'
# answer, context = model.predict(query)
# print(answer)

In [19]:
# query = 'Какие группы оборудования и трубопроводов по НП-089 существуют на атомной станции?'
# answer, context = model.predict(query)
# print(answer)